In [ ]:
%logstop
%logstart -ortq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [4]:
#%%bash
#mkdir data
#wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

In [28]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [6]:
data[0]

{'reviewerID': 'A238V1XTSK9NFE',
 'asin': 'B00004VX3T',
 'reviewerName': 'Andrew Lynn',
 'helpful': [2, 2],
 'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatical

In [29]:
ratings = [d['overall'] for d in data]
ratings[:10]

[5.0, 1.0, 4.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0]

In [8]:
review = [data[i]['reviewText'] for i in range(len(data))]
review[0]

"I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatically supports wheel mice.  Just plug it in and use it - it's that easy."

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

## Custorm --- Json Transformer

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class JsonTrnsformer (BaseEstimator, TransformerMixin):
    """Transform Json to list by given key"""
    def __init__(self,key):
        self.key = key
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [d[self.key] for d in X]


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.pipeline import Pipeline

# Create Pipeline
bag_of_words_model = Pipeline([
    ('review_result', JsonTrnsformer("reviewText")),
    ('bags_of_words',CountVectorizer()),
    ('classfier',Ridge(alpha=0.1))
])

In [11]:
#bag_of_words_model.fit(data,ratings)

In [12]:
#grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from spacy.lang.en import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.pipeline import Pipeline

def lemmatizer(text):
    return [word.lemma_ for word in nlp(text)]



normalized_model = Pipeline([
    ('review_result', JsonTrnsformer("reviewText")),
    ('tfid',TfidfVectorizer(max_features=3000)),
    ('classfier',Ridge(alpha=0.1))
])

In [14]:
normalized_model.fit(data,ratings)

Pipeline(memory=None,
         steps=[('review_result', JsonTrnsformer(key='reviewText')),
                ('tfid',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=3000,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classfier',
                 Ridge(alpha=0.1, copy_X=True, fit_intercept=True,
      

In [15]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score:  0.9991540771622206


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [42]:
from spacy.lang.en import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from spacy.lang.en import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np

gridSearch_ridge = GridSearchCV(Ridge(),param_grid = {"alpha": [0.5, 1. , 1.5, 2. ]},
                  cv = 3,
                  n_jobs =1,
                  verbose = 1
                 )

bigrams_model = Pipeline([
    ('review_result', JsonTrnsformer("reviewText")),
#    ('tfid',TfidfVectorizer(max_features=3000)),
    ('hash',HashingVectorizer(lowercase=True,ngram_range=(1,2), n_features = 20000)),
    ('classfier',gridSearch_ridge)
])

In [43]:
bigrams_model.fit(data,ratings)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   43.1s finished


Pipeline(memory=None,
         steps=[('review_result', JsonTrnsformer(key='reviewText')),
                ('hash',
                 HashingVectorizer(alternate_sign=True, analyzer='word',
                                   binary=False, decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, n_features=20000,
                                   ngram_range=(1, 2), norm='l2',
                                   preprocessor=None, stop_words=None,
                                   strip_accents=None...
                ('classfier',
                 GridSearchCV(cv=3, error_score='raise-deprecating',
                              estimator=Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                           

In [44]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score:  1.0076943288287978


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [45]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

File ‘./data/amazon_one_and_five_star_reviews.json.gz’ already there; not retrieving.



To avoid memory issue, we can delete the older data.

In [46]:
del data, ratings

In [52]:
from sklearn.base import BaseEstimator, TransformerMixin
from spacy.lang.en import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

class JsonTrnsformer (BaseEstimator, TransformerMixin):
    """Transform Json to list by given key"""
    def __init__(self,key):
        self.key = key
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [d[self.key] for d in X]


In [56]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

rating_transform = JsonTrnsformer("overall")
ratings = rating_transform.fit_transform(data_polarity)

polarity_model = Pipeline([
    ('reviews', JsonTrnsformer('reviewText')),
    ('tf_idf',TfidfVectorizer(stop_words = "english")),
    ('Classfier_naive_Bayes', MultinomialNB())
])

In [57]:
polarity_model.fit(data_polarity,ratings)

Pipeline(memory=None,
         steps=[('reviews', JsonTrnsformer(key='reviewText')),
                ('tf_idf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('Classfier_naive_Bayes',
                 MultinomialNB(alpha=1.0, class_prior=None, 

In [59]:
#Get feat from model
feat_to_token = polarity_model.named_steps['tf_idf'].get_feature_names()

# Get proba from model
log_prob = polarity_model['Classfier_naive_Bayes'].feature_log_prob_

# Calculate polarity
polarity = log_prob[0,:]-log_prob[1,:]

# Combine polarity and feat 
most_polar = sorted(list(zip(polarity, feat_to_token)))

# Take the first 25 and the last 25 from most_polar
n = 25
most_polar = most_polar[:n]+most_polar[-n:]

top_50 = [x[1] for x in most_polar]

In [60]:
grader.score.nlp__most_polar(top_50)

Your score:  1.0


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
topic_model = Pipeline([
    ('selector', JsonTrnsformer('reviewText')),
    ('vectorizer', TfidfVectorizer(stop_words='english')), 
    ('reduction', NMF(n_components=20)),
 #   ('classifier', MultinomialNB())
])
topic_model.fit(data_polarity)
nmf = topic_model.named_steps['reduction']
ind_to_token = topic_model.named_steps['vectorizer'].get_feature_names()
words_in_topics = []
for topic in nmf.components_:
    idx = topic.argsort()[-10:]
    top_words = [ind_to_token[i] for i in idx]
    words_in_topics.append(top_words)
 

In [64]:
import pandas as pd
wdict = {}
for i, wlist in enumerate(words_in_topics):
    wdict['Topic # '+'{:02d}'.format(i+1)]= wlist
pd.DataFrame(wdict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,got,wide,music,needed,remote,dvds,perfect,hand,products,volume,sd,link,protection,far,port,car,expected,like,software,length
1,amazon,nikon,headphone,length,flash,tv,use,mice,tech,set,software,firmware,quality,say,use,radios,charm,nice,windows,belkin
2,months,sharp,comfortable,extension,battery,unit,value,used,advertised,bose,quot,internet,lenses,really,plug,better,say,fits,remote,ve
3,money,light,head,modem,batteries,discs,need,buttons,service,surround,sandisk,support,protect,pretty,adapter,station,perfectly,tripod,make,cheap
4,don,hood,bass,belkin,canon,disc,worked,microsoft,purchase,sub,cf,connection,hoya,build,hub,signal,tv,strap,did,connectors
5,buy,50mm,pair,signal,cameras,sony,highly,trackball,does,bass,install,network,tiffen,value,device,stations,cord,small,doesn,monster
6,bought,focus,ears,connect,use,play,easy,use,support,wire,memory,netgear,glass,sound,computer,reception,just,carry,didn,ordered
7,unit,lenses,ear,monitor,digital,cd,recommend,logitech,amazon,speaker,windows,linksys,uv,price,drive,fm,like,fit,tried,hosa
8,time,canon,sound,tv,pictures,player,price,keyboard,recommend,sound,cards,wireless,filters,quality,power,antenna,fine,case,does,quality
9,just,lens,headphones,cable,camera,dvd,great,mouse,product,speakers,card,router,filter,good,usb,radio,works,bag,work,cables


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*